In [147]:
import openrouteservice
import folium
import pandas as pd
import geopandas as gpd
import pyspark
from pyspark.sql import SparkSession, functions as F
import json
import requests
import regex

In [148]:
#f = open("../data/raw/domain_scraped_raw.json")
#rental_data = json.load(f)
#rental_df = pd.DataFrame.from_dict(rental_data, orient = 'index')
rental_df = pd.read_csv("../data/curated/base_rental.csv")

In [194]:
externals = []

In [149]:
rental_df['coordinates'] = rental_df['coordinates'].str.replace(' ', '').str.replace('[', '').str.replace(']', '')
rental_df['coordinates'] = rental_df['coordinates'].str.split(',')

/tmp/ipykernel_1402/2100596331.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rental_df['coordinates'] = rental_df['coordinates'].str.replace(' ', '').str.replace('[', '').str.replace(']', '')


In [150]:
for i in range(0,len(rental_df['coordinates'])):
    rental_df['coordinates'][i] = [float(x) for x in rental_df['coordinates'][i]]

/tmp/ipykernel_1402/4064507194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental_df['coordinates'][i] = [float(x) for x in rental_df['coordinates'][i]]


In [151]:
rental_df.head()

,name,prop_type,cost_text,coordinates,rooms,add_feat
0,817/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,460.0,"[-37.8294033, 144.9692407]","[['2 Beds'], ['1 Bath'], []]","['Air conditioning', 'Built in wardrobes', 'Fu..."
1,1505/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,650.0,"[-37.8294033, 144.9692407]","[['2 Beds'], ['2 Baths'], ['1 Parking']]","['Internal Laundry', 'INSIDE', 'Gym', 'Interco..."
2,3306/151 City Road Southbank VIC 3006,Apartment / Unit / Flat,610.0,"[-37.8236674, 144.9631638]","[['2 Beds'], ['2 Baths'], ['1 Parking']]","['Gas', 'Internal Laundry', 'Heating', 'Secure..."
3,168/100 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,480.0,"[-37.8251485, 144.9627575]","[['1 Bed'], ['1 Bath'], ['1 Parking']]","['Gas', 'Internal Laundry', 'Intercom', 'Secur..."
4,1412/151 City Road Southbank VIC 3006,Apartment / Unit / Flat,620.0,"[-37.8236674, 144.9631638]","[['2 Beds'], ['2 Baths'], ['1 Parking']]","['Gas', 'Internal Laundry', 'Secure Parking', ..."


In [152]:
# Makes all the coordinates scraped into (lattitude ,longitude)--> its actually (longitude, lattitude)
rental_df['coordinates'] = rental_df['coordinates'].apply(lambda x: list(reversed(x)))

In [153]:
rental_df = rental_df.sample(500)

In [154]:
rental_df.head()

,name,prop_type,cost_text,coordinates,rooms,add_feat
1550,165 Brunswick Road Brunswick VIC 3056,House,725.0,"[144.9664108, -37.7775899]","[['3 Beds'], ['1 Bath'], []]","['Built in wardrobes', 'Gas', 'Internal Laundr..."
5015,13 Rhapsody Road Beveridge VIC 3753,House,450.0,"[144.9428402, -37.4759712]","[['4 Beds'], ['2 Baths'], ['2 Parking']]","['Air conditioning', 'Gas', 'Internal Laundry'..."
2472,5/56-58 Burwood Road Hawthorn VIC 3122,Apartment / Unit / Flat,375.0,"[145.0241732, -37.8214991]","[['1 Bed'], ['1 Bath'], ['1 Parking']]",['Dishwasher']
4082,2/10 Ashmore Avenue Mordialloc VIC 3195,Apartment / Unit / Flat,340.0,"[145.0863488, -38.0012145]","[['1 Bed'], ['1 Bath'], ['1 Parking']]",['No Extra Features Listed']
1010,21 Bartek Street Tarneit VIC 3029,House,460.0,"[144.6402182, -37.8553075]","[['4 Beds'], ['2 Baths'], ['2 Parking']]","['Secure Parking', 'Heating', 'Dishwasher']"


In [155]:
len(rental_df)

500

# Distance from Melbourne CBD

In [156]:
# create list:
cbd_dur = []
cbd_dist = []

In [157]:
client = openrouteservice.Client(base_url="http://localhost:8080/ors")

In [158]:
for i in range(len(rental_df)):
    #print(test.iloc[i,2])
    body = {"coordinates":[rental_df.iloc[i,3],[144.963058,-37.813629]],"instructions":"false","units":"km",'radiuses':[500]}
    headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Content-Type': 'application/json; charset=utf-8'
    }
    call = requests.post('http://localhost:8080/ors/v2/directions/driving-car/json', json=body, headers=headers)
    response_dict = json.loads(call.text)
    # Saves the distance and duration from melbourne cbd if there are routes available, if not note it down using negative one
    if 'routes' in response_dict.keys():
        # need to be able to fix warning setting w coopy warning
        cbd_dur.append(response_dict['routes'][0]['summary']['duration']/60)
        cbd_dist.append(response_dict['routes'][0]['summary']['distance']  )
    else:
        cbd_dur.append(-1)
        cbd_dist.append(-1)



In [201]:
cbd_dur_df = pd.DataFrame(cbd_dur, columns = ['train_prox_2km'])
cbd_dist_df = pd.DataFrame(cbd_dist, columns = ['train_prox_2_min'])

In [223]:
rental_df['cbd_dur'] = cbd_dur
rental_df['cbd_dist'] = cbd_dist

In [224]:
rental_df.head()

,name,prop_type,cost_text,coordinates,rooms,add_feat,cbd_dur,cbd_dist
1550,165 Brunswick Road Brunswick VIC 3056,House,725.0,"[144.9664108, -37.7775899]","[['3 Beds'], ['1 Bath'], []]","['Built in wardrobes', 'Gas', 'Internal Laundr...",7.925000,4.778
5015,13 Rhapsody Road Beveridge VIC 3753,House,450.0,"[144.9428402, -37.4759712]","[['4 Beds'], ['2 Baths'], ['2 Parking']]","['Air conditioning', 'Gas', 'Internal Laundry'...",53.136667,47.784
2472,5/56-58 Burwood Road Hawthorn VIC 3122,Apartment / Unit / Flat,375.0,"[145.0241732, -37.8214991]","[['1 Bed'], ['1 Bath'], ['1 Parking']]",['Dishwasher'],11.945000,6.243
4082,2/10 Ashmore Avenue Mordialloc VIC 3195,Apartment / Unit / Flat,340.0,"[145.0863488, -38.0012145]","[['1 Bed'], ['1 Bath'], ['1 Parking']]",['No Extra Features Listed'],43.720000,27.956
1010,21 Bartek Street Tarneit VIC 3029,House,460.0,"[144.6402182, -37.8553075]","[['4 Beds'], ['2 Baths'], ['2 Parking']]","['Secure Parking', 'Heating', 'Dishwasher']",37.708333,34.878


# Distance and duration from train


In [161]:
# test if can use poi
from shapely.geometry import Point

In [162]:
sf = gpd.read_file("../data/raw/TRANSPORT/VIC_RAILWAY_STATIONS.shp")

In [163]:
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf.head()

,STATION,STATUS,STOPID_MTR,STID_VLINE,PREMIUM,METRO,VLINE,INTERSTATE,STOP_ZONE,COMMENTS,geometry
0,Dandenong,Active,19888,22250,Yes,Yes,Yes,No,"Reg, 2",Metro Interchange,POINT (145.20964 -37.98992)
1,Darebin,Active,19932,None,No,Yes,No,No,1,None,POINT (145.03863 -37.77480)
2,Darling,Active,19837,None,Yes,Yes,No,No,"1, 2",None,POINT (145.06268 -37.86867)
3,Deer Park,Active,None,19982,No,No,Yes,No,"Reg, 2",None,POINT (144.77122 -37.77750)
4,Dennis,Active,19929,None,No,Yes,No,No,1,None,POINT (145.00871 -37.77918)


In [164]:
#Only keep active stations
active_stations = sf.loc[sf['STATUS'] == 'Active']
active_stations.count()

STATION       314
STATUS        314
STOPID_MTR    220
STID_VLINE    114
PREMIUM       311
METRO         314
VLINE         314
INTERSTATE    314
STOP_ZONE     310
COMMENTS       50
geometry      314
dtype: int64

In [165]:
#find closest coordinate
sf['long'] = sf['geometry'].x
sf['lat'] = sf['geometry'].y

In [166]:
#find closest coordinate
sf['long'] = sf['geometry'].x
sf['lat'] = sf['geometry'].y

In [167]:
sf

,STATION,STATUS,STOPID_MTR,STID_VLINE,PREMIUM,METRO,VLINE,INTERSTATE,STOP_ZONE,COMMENTS,geometry,long,lat
0,Dandenong,Active,19888,22250,Yes,Yes,Yes,No,"Reg, 2",Metro Interchange,POINT (145.20964 -37.98992),145.209645,-37.989921
1,Darebin,Active,19932,None,No,Yes,No,No,1,None,POINT (145.03863 -37.77480),145.038628,-37.774803
2,Darling,Active,19837,None,Yes,Yes,No,No,"1, 2",None,POINT (145.06268 -37.86867),145.062679,-37.868670
3,Deer Park,Active,None,19982,No,No,Yes,No,"Reg, 2",None,POINT (144.77122 -37.77750),144.771221,-37.777504
4,Dennis,Active,19929,None,No,Yes,No,No,1,None,POINT (145.00871 -37.77918),145.008712,-37.779180
...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,Cranbourne,Active,19886,None,Yes,Yes,No,No,2,Metro Terminus,POINT (145.28001 -38.09916),145.280013,-38.099164
312,Creswick,Active,None,44951,No,No,Yes,No,"Reg, 9",None,POINT (143.88816 -37.42448),143.888160,-37.424484
313,Crib Point,Active,19828,None,No,Yes,No,No,2,None,POINT (145.20398 -38.36593),145.203977,-38.365926
314,Croxton,Active,20015,None,No,Yes,No,No,1,None,POINT (144.99710 -37.76384),144.997105,-37.763841


# Use Points of interest to determine how many there are in 2 km radius?


In [168]:
# Take all coordinates and create a list of it for geojson input:
# sf.iloc[i,11],sf.iloc[i,12] are longitude and latittude of stations
train_coords = []
for i in range(len(sf)):
    train_coords.append([sf.iloc[i,11],sf.iloc[i,12]])

In [169]:
len_sources = len(train_coords) 

In [170]:
len_sources

316

In [171]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Content-Type': 'application/json; charset=utf-8'
}

In [172]:
base_body = {'locations':train_coords,'metrics':('duration','distance'),'units':'km', 'sources':[len_sources]}


In [173]:
len(base_body['locations'])


316

In [174]:
len(train_coords)

316

In [202]:
train_prox_2km = []
train_prox_2_min = []
train_prox_closest = []

In [203]:
# Tests whether or not there are train stations within 2 km and 5 minutes of driving time 
for i in range(len(rental_df)):
    base_body['locations'].append(rental_df.iloc[i,3])
    call = requests.post('http://localhost:8080/ors/v2/matrix/driving-car', json=base_body, headers=headers)
    response_dict = json.loads(call.text)
    if 'distances' in response_dict.keys():
        train_prox_2km.append(sum(each_dist <= 2 for each_dist in response_dict['distances'][0]) - 1 )
        train_prox_2_min.append(sum(each_dur <= 120 for each_dur in response_dict['durations'][0]) - 1 )
        train_prox_closest.append(min([dist for dist in response_dict['distances'][0] if dist!=0]))
    else:
        train_prox_2km.append(-1)
        train_prox_2_min.append(-1)
        train_prox_closest.append(-1)
    base_body['locations'].pop()

In [204]:
train_prox_2km_df = pd.DataFrame(train_prox_2km, columns = ['train_prox_2km'])
train_prox_2_min_df = pd.DataFrame(train_prox_2_min, columns = ['train_prox_2_min'])
train_prox_closest_df = pd.DataFrame(train_prox_closest, columns = ['train_prox_closest'])

In [225]:
rental_df['train_prox_2km'] = train_prox_2km
rental_df['train_prox_2_min'] = train_prox_2_min
rental_df['train_prox_closest'] = train_prox_closest


In [226]:
rental_df.head()

,name,prop_type,cost_text,coordinates,rooms,add_feat,cbd_dur,cbd_dist,train_prox_2km,train_prox_2_min,train_prox_closest
1550,165 Brunswick Road Brunswick VIC 3056,House,725.0,"[144.9664108, -37.7775899]","[['3 Beds'], ['1 Bath'], []]","['Built in wardrobes', 'Gas', 'Internal Laundr...",7.925000,4.778,8,2,0.75
5015,13 Rhapsody Road Beveridge VIC 3753,House,450.0,"[144.9428402, -37.4759712]","[['4 Beds'], ['2 Baths'], ['2 Parking']]","['Air conditioning', 'Gas', 'Internal Laundry'...",53.136667,47.784,0,0,3.94
2472,5/56-58 Burwood Road Hawthorn VIC 3122,Apartment / Unit / Flat,375.0,"[145.0241732, -37.8214991]","[['1 Bed'], ['1 Bath'], ['1 Parking']]",['Dishwasher'],11.945000,6.243,9,3,0.37
4082,2/10 Ashmore Avenue Mordialloc VIC 3195,Apartment / Unit / Flat,340.0,"[145.0863488, -38.0012145]","[['1 Bed'], ['1 Bath'], ['1 Parking']]",['No Extra Features Listed'],43.720000,27.956,6,1,0.64
1010,21 Bartek Street Tarneit VIC 3029,House,460.0,"[144.6402182, -37.8553075]","[['4 Beds'], ['2 Baths'], ['2 Parking']]","['Secure Parking', 'Heating', 'Dishwasher']",37.708333,34.878,0,0,2.51


In [196]:
#train_prox_2km_df.to_csv("../data/raw/train_prox_2km_draft.csv")

In [179]:
train_prox_2km_df.value_counts()

train_prox_2km
0                 206
1                 103
2                  90
3                  69
4                  24
5                   6
6                   2
dtype: int64

In [180]:
train_prox_5_min.value_counts()

train_prox_2_min
0                   396
1                    90
2                    14
dtype: int64

# Now the same for Schools

In [181]:
aus_school = pd.read_csv("../data/raw/australia_school.csv",encoding='cp1252')

In [182]:
vic_school = aus_school[aus_school['Postal_State'] == 'VIC']

In [183]:
vic_school.head(5)

,Education_Sector,Entity_Type,SCHOOL_NO,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,...,Postal_Address_Line_1,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y
0,Government,1,1,Alberton Primary School,Primary,O,21 Thomson Street,NaN,Alberton,VIC,...,21 Thomson Street,NaN,ALBERTON,VIC,3971,03 5183 2412,681,Wellington (S),146.666601,-38.617713
1,Government,1,3,Allansford and District Primary School,Primary,O,Frank Street,NaN,Allansford,VIC,...,Frank Street,NaN,ALLANSFORD,VIC,3277,03 5565 1382,673,Warrnambool (C),142.590393,-38.386281
2,Government,1,4,Avoca Primary School,Primary,O,118 Barnett Street,NaN,Avoca,VIC,...,P O Box 12,NaN,AVOCA,VIC,3467,03 5465 3176,599,Pyrenees (S),143.475649,-37.084502
3,Government,1,8,Avenel Primary School,Primary,O,40 Anderson Street,NaN,Avenel,VIC,...,40 Anderson Street,NaN,AVENEL,VIC,3664,03 5796 2264,643,Strathbogie (S),145.234722,-36.901368
4,Government,1,12,Warrandyte Primary School,Primary,O,5-11 Forbes Street,NaN,Warrandyte,VIC,...,5-11 Forbes Street,NaN,WARRANDYTE,VIC,3113,03 9844 3537,421,Manningham (C),145.213980,-37.742675


In [184]:
# remember that it is longitude first then lattitude: y and then x
vic_school_coord = []
school_long = vic_school['X'].tolist()
school_lat = vic_school['Y'].tolist()

In [185]:
for i in range(0,len(school_long)):
    vic_school_coord.append([school_long[i],school_lat[i]])

In [186]:
base_body = {'locations':vic_school_coord,'metrics':('duration','distance'),'units':'km', 'sources':[len(vic_school_coord)]}
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Content-Type': 'application/json; charset=utf-8'
}

In [205]:
school_prox_15km = []
school_prox_2_min = []
school_prox_closest = []

In [206]:
# Tests whether or not there are school within 2 km and 5 minutes of driving time, the problem was that only allowed 350 routes: later on change it so that takes closest few
for i in range(len(rental_df)):
    base_body['locations'].append(rental_df.iloc[i,3])
    call = requests.post('http://localhost:8080/ors/v2/matrix/driving-car', json=base_body, headers=headers)
    response_dict = json.loads(call.text)
    if 'distances' in response_dict.keys():
        school_prox_15km.append(sum(each_dist <= 10 for each_dist in response_dict['distances'][0]) - 1 )
        school_prox_2_min.append(sum(each_dur <= 120 for each_dur in response_dict['durations'][0]) - 1 )
        school_prox_closest.append(min([dist for dist in response_dict['distances'][0] if dist!=0]))
    else:
        school_prox_15km.append(-1)
        school_prox_2_min.append(-1)
        school_prox_closest.append(-1)
    base_body['locations'].pop()

In [211]:
response_dict

{'durations': [[9421.52,
   11483.2,
   8420.7,
   6194.84,
   2233.02,
   4537.01,
   3282.58,
   5545.66,
   13400.31,
   2031.07,
   7152.99,
   1439.5,
   11537.25,
   1521.18,
   13416.37,
   7082.79,
   6513.03,
   6757.29,
   5778.96,
   7992.84,
   3496.9,
   12182.18,
   3354.17,
   1358.82,
   11060.07,
   1988.52,
   8139.13,
   5877.55,
   1364.34,
   1463.94,
   1672.07,
   3815.88,
   1291.19,
   14607.03,
   4675.46,
   518.13,
   1318.26,
   13520.21,
   6771.35,
   4364.8,
   8714.19,
   5101.72,
   12216.47,
   4912.86,
   4706.84,
   5742.78,
   6996.02,
   7601.78,
   4184.99,
   5845.7,
   2898.76,
   1113.4,
   7309.99,
   5673.32,
   1032.4,
   3882.18,
   7171.12,
   1811.45,
   1783.61,
   12394.68,
   3615.31,
   3128.16,
   16470.74,
   10559.29,
   12216.55,
   4326.85,
   7311.5,
   4778.27,
   9518.81,
   8178.83,
   509.4,
   9999.97,
   5777.26,
   12497.18,
   4980.16,
   2192.38,
   10914.65,
   12400.5,
   4507.81,
   11926.87,
   2492.77,
   6730.43,

In [208]:
#Theres a problem with 15 km
test= pd.DataFrame(school_prox_15km)

In [209]:
test.value_counts()

155    9
162    8
186    7
18     7
132    6
      ..
157    1
123    1
66     1
56     1
103    1
Length: 178, dtype: int64

In [212]:
school_prox_15km_df = pd.DataFrame(school_prox_15km, columns = ['school_prox_15km'])
school_prox_2_min_df = pd.DataFrame(school_prox_2_min, columns = ['school_prox_2_min'])
school_prox_closest_df = pd.DataFrame(school_prox_closest, columns = ['school_prox_closest'])

In [53]:
#for some reason its always the same
#school_prox_closest

In [227]:
rental_df['school_prox_15km'] = school_prox_15km
rental_df['school_prox_2_min'] = school_prox_2_min
rental_df['school_prox_closest'] = school_prox_closest


In [232]:
rental_df.head()

,name,prop_type,cost_text,coordinates,rooms,add_feat,cbd_dur,cbd_dist,train_prox_2km,train_prox_2_min,train_prox_closest,school_prox_15km,school_prox_2_min,school_prox_closest
1550,165 Brunswick Road Brunswick VIC 3056,House,725.0,"[144.9664108, -37.7775899]","[['3 Beds'], ['1 Bath'], []]","['Built in wardrobes', 'Gas', 'Internal Laundr...",7.925000,4.778,8,2,0.75,178,2,0.75
5015,13 Rhapsody Road Beveridge VIC 3753,House,450.0,"[144.9428402, -37.4759712]","[['4 Beds'], ['2 Baths'], ['2 Parking']]","['Air conditioning', 'Gas', 'Internal Laundry'...",53.136667,47.784,0,0,3.94,2,0,3.94
2472,5/56-58 Burwood Road Hawthorn VIC 3122,Apartment / Unit / Flat,375.0,"[145.0241732, -37.8214991]","[['1 Bed'], ['1 Bath'], ['1 Parking']]",['Dishwasher'],11.945000,6.243,9,3,0.37,190,3,0.37
4082,2/10 Ashmore Avenue Mordialloc VIC 3195,Apartment / Unit / Flat,340.0,"[145.0863488, -38.0012145]","[['1 Bed'], ['1 Bath'], ['1 Parking']]",['No Extra Features Listed'],43.720000,27.956,6,1,0.64,58,1,0.64
1010,21 Bartek Street Tarneit VIC 3029,House,460.0,"[144.6402182, -37.8553075]","[['4 Beds'], ['2 Baths'], ['2 Parking']]","['Secure Parking', 'Heating', 'Dishwasher']",37.708333,34.878,0,0,2.51,37,0,2.51


In [233]:
rental_df.to_csv("../data/curated/external_rental.csv", index = False)

# Now for Parks

In [2]:
# park shapefile downloaded from: Parks and Conservation Reserves (PARKRES) - Dataset - Victorian Government Data Directory
# recreational locations from:https://discover.data.vic.gov.au/dataset/sport-and-recreational-facilities-list, renamed recre_fac

In [66]:
recre_fal = pd.read_excel("../data/raw/recre_fal.xlsx")

In [72]:
recre_fal['Latitude']

0      -37.851683
1      -37.851683
2      -37.851683
3      -37.851683
4      -37.851683
          ...    
9585   -35.346917
9586   -35.346917
9587   -35.683695
9588   -35.767453
9589   -36.252513
Name: Latitude, Length: 9590, dtype: float64

In [67]:
recre_fal.columns

Index(['Facility ID', 'LGA Name', 'Facility Name', 'Facility_AutoNumber',
       'Street #', 'Street Name', 'Street Type', 'Suburb/Town', 'Pcode',
       'Melway Ref', 'VicRoads Ref', 'Latitude', 'Longitude',
       'Facility Ownership', 'Facility Purpose', 'Facility Category',
       'CFA Safer Place?', 'Access_', 'FaciltySportPlayedID', 'Facility ID.1',
       'Sports Played', 'Number of Field/Courts', 'Field/Surface Type',
       'Age of Facility', 'Condition of Facility', 'Facility Upgrade Age',
       'Changerooms', 'Facility Features',
       'Spectator numbers for seating/shelter', 'FullAddress',
       'MelwaysVicRoadsRef'],
      dtype='object')

In [73]:
keep = ['Suburb/Town', 'Pcode','Latitude', 'Longitude']
recre_fal_df = pd.DataFrame(recre_fal,columns= keep)

In [106]:
recre_fal_df[recre_fal_df['Longitude'] == 'NaN']

,Suburb/Town,Pcode,Latitude,Longitude


In [75]:
# remember that it is longitude first then lattitude: y and then x
recre_coord = []
recre_long = recre_fal_df['Longitude'].tolist()
recre_lat = recre_fal_df['Latitude'].tolist()

In [76]:
for i in range(0,len(recre_long)):
    recre_coord.append([recre_long[i],recre_lat[i]])

In [87]:
recre_prox_15km = []
recre_prox_2_min = []
recre_prox_closest = []

In [95]:
len(recre_coord)

9590

In [99]:
base_body = {'locations':recre_coord,'metrics':('duration','distance'),'units':'km', 'sources':[len(recre_coord)]}
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Content-Type': 'application/json; charset=utf-8'
}

In [110]:
# Tests whether or not there are school within 2 km and 5 minutes of driving time, the problem was that only allowed 350 routes: later on change it so that takes closest few
for i in range(len(rental_df)):
    base_body['locations'].append(rental_df.iloc[i,3])
    call = requests.post('http://localhost:8080/ors/v2/matrix/driving-car', json=base_body, headers=headers)
    response_dict = json.loads(call.text)
    if 'distances' in response_dict.keys():
        recre_prox_15km.append(sum(each_dist <= 15 for each_dist in response_dict['distances'][0]) - 1 )
        recre_prox_2_min.append(sum(each_dur <= 120 for each_dur in response_dict['durations'][0]) - 1 )
        recre_prox_closest.append(min([dist for dist in response_dict['distances'][0] if dist!=0]))
    else:
        recre_prox_15km.append(-1)
        recre_prox_2_min.append(-1)
        recre_prox_closest.append(-1)
    base_body['locations'].pop()

In [111]:
response_dict

{'error': {'message': 'JSON parse error: Non-standard token \'NaN\': enable JsonParser.Feature.ALLOW_NON_NUMERIC_NUMBERS to allow; nested exception is com.fasterxml.jackson.databind.JsonMappingException: Non-standard token \'NaN\': enable JsonParser.Feature.ALLOW_NON_NUMERIC_NUMBERS to allow\n at [Source: (PushbackInputStream); line: 1, column: 1032] (through reference chain: org.heigit.ors.api.requests.matrix.MatrixRequest["locations"]->java.util.ArrayList[39])'},
 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-10-04T21:13:01Z'},
  'timestamp': 1664922598067}}

# Now for hospitals